In [1]:
using Distributed
@everywhere using StaticArrays,BenchmarkTools,Random,Flux,DelimitedFiles,LinearAlgebra
@everywhere using InvertedIndices
@everywhere using MachineLearningPotential

# Symmetry Functions

Time to write and test the basics of SymmFunctions for ML potentials

In [2]:
atoms = [[0.0000006584,       -0.0000019175,        0.0000000505],
[-0.0000005810,       -0.0000004871,        0.6678432175],
[0.1845874248,       -0.5681026047,        0.2986701538],
[-0.4832557457,       -0.3511072166,        0.2986684497],
[-0.4832557570,        0.3511046452,        0.2986669456],
[0.1845874064,        0.5681000550,        0.2986677202],
[0.5973371920,       -0.0000012681,        0.2986697030],
[-0.1845860897,       -0.5681038901,       -0.2986676192],
[-0.5973358752,       -0.0000025669,       -0.2986696020],
[-0.1845861081,        0.5680987696,       -0.2986700528],
[0.4832570624,        0.3511033815,       -0.2986683486],
[0.4832570738,       -0.3511084803,       -0.2986668445],
[0.0000018978,       -0.0000033480,       -0.6678431165],
[-0.0000017969,        0.0000009162,        1.3230014650],
[0.1871182835,       -0.5758942175,        0.9797717078],
[-0.4898861924,       -0.3559221410,       0.9797699802],
[-0.4898862039,        0.3559224872,        0.9797684555],
[0.1871182648,        0.5758945856,        0.9797692407],
[0.6055300485,        0.0000001908,        0.9797712507],
[0.7926501864,       -0.5758950093,        0.6055339635],
[0.3656681761,       -1.1254128670,        0.5916673591],
[-0.3027660545,       -0.9318173412,        0.6055326929],
[-0.9573332453,       -0.6955436707,        0.5916639831],
[-0.9797705418,       -0.0000006364,        0.6055294407],
[-0.9573332679,        0.6955423392,        0.5916610035],
[-0.3027660847,        0.9318160902,        0.6055287012],
[0.3656681396,        1.1254115783,        0.5916625380],
[0.7926501677,        0.5758937939,        0.6055314964],
[1.1833279992,       -0.0000006311,        0.5916664660],
[0.6770051458,       -0.9318186223,        0.0000033028],
[0.0000006771,       -1.1517907207,        0.0000025175],
[-0.6770037988,       -0.9318186442,        0.0000007900],
[-1.0954155825,       -0.3559242494,       -0.0000012200],
[-1.0954155940,        0.3559203788,       -0.0000027447],
[-0.6770038290,        0.9318147872,       -0.0000032017],
[0.0000006397,        1.1517868856,       -0.0000024165],
[0.6770051155,        0.9318148091,       -0.0000006889],
[1.0954168993,        0.3559204143,        0.0000013211],
[1.0954169108,       -0.3559242139,        0.0000028458],
[0.3027674014,       -0.9318199253,       -0.6055286002],
[-0.3656668229,       -1.1254154134,       -0.5916624370],
[-0.7926488510,       -0.5758976290,       -0.6055313954],
[-1.1833266824,       -0.0000032040,       -0.5916663649],
[-0.7926488697,        0.5758911742,       -0.6055338624],
[-0.3656668594,        1.1254090319,       -0.5916672580],
[0.3027673712,        0.9318135061,       -0.6055325919],
[0.9573345621,        0.6955398357,       -0.5916638820],
[0.9797718586,       -0.0000031986,       -0.6055293396],
[0.9573345846,       -0.6955461743,       -0.5916609025],
[-0.1871169480,       -0.5758984207,       -0.9797691397],
[-0.6055287318,       -0.0000040259,       -0.9797711497],
[-0.1871169667,        0.5758903824,       -0.9797716067],
[0.4898875091,        0.3559183059,       -0.9797698792],
[0.4898875207,       -0.3559263223,       -0.9797683545],
[0.0000031136,       -0.0000047513,       -1.3230013639]]*18.8973*0.36258

positions = [SVector{3}(p[i] for i in 1:3) for p in atoms]
dis2mat = get_distance2_mat(positions)
rad_function = RadialType2{Float64}(0.001,11.338,[1.,1.])
ang_func = AngularType3{Float64}(0.0001,1.0,1.0,11.338,[1.,1.,1.])
ang_func2 = AngularType3{Float64}(0.0001,-1.0,2.0,11.338,[1.,1.,1.])
X = [ 1    1              0.001   0.000  11.338
 1    0              0.001   0.000  11.338
 1    1              0.020   0.000  11.338
 1    0              0.020   0.000  11.338
 1    1              0.035   0.000  11.338
 1    0              0.035   0.000  11.338
 1    1              0.100   0.000  11.338
 1    0              0.100   0.000  11.338
 1    1              0.400   0.000  11.338
 1    0              0.400   0.000  11.338]

radsymmvec = []

for row in eachrow(X)
    symmfunc = RadialType2{Float64}(row[3],row[5],[row[1],row[2]])
    push!(radsymmvec,symmfunc)
end

V = [[0.0001,1,1,11.338],[0.0001,-1,2,11.338],[0.003,-1,1,11.338],[0.003,-1,2,11.338],[0.008,-1,1,11.338],[0.008,-1,2,11.338],[0.008,1,2,11.338],[0.015,1,1,11.338],[0.015,-1,2,11.338],[0.015,-1,4,11.338],[0.015,-1,16,11.338],[0.025,-1,1,11.338],[0.025,1,1,11.338],[0.025,1,2,11.338],[0.025,-1,4,11.338],[0.025,-1,16,11.338],[0.025,1,16,11.338],[0.045,1,1,11.338],[0.045,-1,2,11.338],[0.045,-1,4,11.338],[0.045,1,4,11.338],[0.045,1,16,11.338],[0.08,1,1,11.338],[0.08,-1,2,11.338],[0.08,-1,4,11.338],[0.08,1,4,11.338]]

T = [[1.,1.,1.],[1.,1.,0.],[1.,0.,0.]]

angularsymmvec = []

for element in V 
    for types in T
        symmfunc = AngularType3{Float64}(element[1],element[2],element[3],11.338,types)
        push!(angularsymmvec,symmfunc)
    end
end

total_symm_vec = vcat(radsymmvec,angularsymmvec)
f_mat = cutoff_function.(sqrt.(dis2mat),Ref(total_symm_vec[1].r_cut))

55×55 Matrix{Float64}:
 1.0        0.649134    0.649134    …  0.21115     0.21115     0.0959383
 0.649134   1.0         0.617699       0.0         0.0         0.0
 0.649134   0.617699    1.0            0.00208061  0.0912928   0.0
 0.649134   0.617699    0.617699       0.0         0.00208061  0.0
 0.649134   0.617699    0.223352       0.00208061  0.0         0.0
 0.649134   0.617699    0.223352    …  0.0912928   0.00208061  0.0
 0.649134   0.617699    0.617699       0.0912928   0.0912928   0.0
 0.649134   0.223352    0.617699       0.0912928   0.355548    0.185434
 0.649134   0.223352    0.223352       0.0912928   0.0912928   0.185434
 0.649134   0.223352    0.0889637      0.355548    0.0912928   0.185434
 ⋮                                  ⋱                          
 0.0959383  0.0         0.0            0.623382    0.164805    0.0613471
 0.21115    0.00208061  0.0912928      0.608792    0.608792    0.164805
 0.0959383  0.0         0.185434       0.164805    0.623382    0.0613471
 0.2

In [3]:
gmat = total_symm_calc(positions,dis2mat,f_mat,total_symm_vec)
@benchmark total_symm_calc($positions,$dis2mat,$f_mat,$total_symm_vec)

BenchmarkTools.Trial: 117 samples with 1 evaluation.
 Range (min … max):  39.893 ms … 50.842 ms  ┊ GC (min … max): 10.06% … 7.67%
 Time  (median):     41.112 ms              ┊ GC (median):     9.66%
 Time  (mean ± σ):   42.813 ms ±  2.880 ms  ┊ GC (mean ± σ):  11.76% ± 3.31%

    ▁█▃█▁                                                      
  ▅▄█████▅▅▅▄▃▃▁▃▄▁▄▄▃▃▄▃▄▁▃▁▁▁▁▁▅▃▃▃▃▃▃▃▄▃▄▃▃▅▁▃▃▃▅▃▄▄▁▁▁▁▁▃ ▃
  39.9 ms         Histogram: frequency by time        49.5 ms <

 Memory estimate: 104.19 MiB, allocs estimate: 1364455.

In [4]:
file = open("$(pwd())/symfunctions.out","r+")
truevalues = readdlm(file)
close(file)
truevals = transpose(truevalues[2:end,2:end])
truevals ≈ gmat

# X=gmat.-truevals

true

# Testing the waters with iterating over symmetry functions

NB This method failed. It was significantly slower due to the iterators and memory access.

In [5]:
perturbation = [-.1 , .3, .2 ]
newpos = positions[3] .+ perturbation
index = 3

new_dis_vec = [distance2(newpos,b) for b in positions]
new_dis_vec[3] = 0.

new_f_vec = cutoff_function.(sqrt.(new_dis_vec),Ref(total_symm_vec[1].r_cut))


55-element Vector{Float64}:
 0.6729622810620872
 0.6673537611763615
 1.0
 0.641313952432365
 0.25867027813036586
 0.25848314873322153
 0.6407783065380429
 0.6000753639415433
 0.23563090780103746
 0.1043164272069067
 ⋮
 0.0
 0.08540286302967126
 0.15905660612244632
 0.0780399503184917
 0.0019532999653086303
 0.0
 0.0019323260080338356
 0.07794636305290437
 0.0

In [6]:
gmat = total_symm_calc(positions,newpos,dis2mat,new_dis_vec,f_mat,new_f_vec,gmat,3,total_symm_vec)


88×55 Matrix{Float64}:
 14.6651      12.0174      12.1633      …  7.44286      5.69954
  0.0          0.0          0.0            0.0          0.0
  7.20013      6.42359      6.44734        3.92544      2.94156
  0.0          0.0          0.0            0.0          0.0
  4.55465      4.2034       4.18932        2.54479      1.91555
  0.0          0.0          0.0         …  0.0          0.0
  0.990147     0.889324     0.869478       0.50839      0.410771
  0.0          0.0          0.0            0.0          0.0
  0.00194292   0.00137073   0.00141713     0.000528536  0.000557398
  0.0          0.0          0.0            0.0          0.0
  ⋮                                     ⋱               
  0.00820484   0.00610573   0.00565103     0.00192164   0.00165975
  0.0          0.0          0.0         …  0.0          0.0
  0.0          0.0          0.0            0.0          0.0
  0.00147572   0.00130362   0.00101334     0.000182034  0.00013714
  0.0          0.0          0.0          

In [7]:
@benchmark total_symm_calc($positions,$newpos,$dis2mat,$new_dis_vec,$f_mat,$new_f_vec,$gmat,$3,$total_symm_vec)

BenchmarkTools.Trial: 544 samples with 1 evaluation.
 Range (min … max):  6.318 ms … 16.957 ms  ┊ GC (min … max):  0.00% … 26.30%
 Time  (median):     7.306 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   9.189 ms ±  2.883 ms  ┊ GC (mean ± σ):  16.44% ± 16.18%

    ▃▂ ▂█▆                                            ▁▃      
  ▃▅██████▆▃▂▃▃▂▃▃▃▂▂▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▃▃▃▃▅██▅▄▃▃ ▃
  6.32 ms        Histogram: frequency by time          14 ms <

 Memory estimate: 28.46 MiB, allocs estimate: 372376.

In [8]:
function testspeed(positions,new_pos,dist2_mat,new_dist_vec,f_mat,new_f_vec,g_mat,atomindex,total_symm_vec)
    for i in 1:100
        total_symm_calc(positions,new_pos,dist2_mat,new_dist_vec,f_mat,new_f_vec,g_mat,atomindex,total_symm_vec)
    end
end
@benchmark testspeed($positions,$newpos,$dis2mat,$new_dis_vec,$f_mat,$new_f_vec,$gmat,$3,$total_symm_vec)

BenchmarkTools.Trial: 6 samples with 1 evaluation.
 Range (min … max):  904.334 ms … 924.005 ms  ┊ GC (min … max): 16.11% … 16.21%
 Time  (median):     913.814 ms               ┊ GC (median):    16.51%
 Time  (mean ± σ):   913.941 ms ±   6.432 ms  ┊ GC (mean ± σ):  16.46% ±  0.25%

  █                    █      █ █      █                      █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁█▁█▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  904 ms           Histogram: frequency by time          924 ms <

 Memory estimate: 2.78 GiB, allocs estimate: 37237600.

In [9]:
@benchmark all_angular_measure($positions[1],$positions[2],$positions[5],$dis2mat[1,2],$dis2mat[1,5],dis2mat[2,5])

BenchmarkTools.Trial: 10000 samples with 990 evaluations.
 Range (min … max):  42.328 ns …   6.146 μs  ┊ GC (min … max):  0.00% … 98.72%
 Time  (median):     47.034 ns               ┊ GC (median):     0.00%
 Time  (mean ± σ):   63.630 ns ± 282.810 ns  ┊ GC (mean ± σ):  22.17% ±  4.94%

   ▄▆▇████▇▆▆▅▄▄▃▂▂▂▂▂▁▁▂▁▁▂▂▂▂▂▂▁▁▁▁ ▁                        ▃
  ████████████████████████████████████████▇▇▇▇█▇▇▇█▆▇▆▅▅▆▆▇▆▇▇ █
  42.3 ns       Histogram: log(frequency) by time        81 ns <

 Memory estimate: 224 bytes, allocs estimate: 7.

# A new approach

Given the obvious limitations of the current approach and the issues previously faced, we acknowledge that three times the original vectors are generated at each step, this causes major problems with memory allocation and garbage collection, resulting in slower calculation of symmetry funcitons overall. 

The aim of the next section is therefore to rewrite the code in a way that modifies existing arrays in place in an effort to reduce allocation overheads. 

In [10]:
exponential_part(η,r2_ij,r2_ik,r2_jk,f_ij,f_ik,f_jk) = exp(-η*(r2_ij+r2_ik+r2_jk))* f_ij * f_ik * f_jk
theta_part(θ,λ,ζ) = (1+λ*θ)^ζ

theta_part (generic function with 1 method)

In [11]:
angsymmvec = []
for symmfunc in angularsymmvec
    values = [symmfunc.eta,symmfunc.lambda,symmfunc.zeta,symmfunc.tpz]
    push!(angsymmvec,values)
end
angsymmvec

78-element Vector{Any}:
 [0.0001, 1.0, 1.0, 1.0]
 [0.0001, 1.0, 1.0, 1.0]
 [0.0001, 1.0, 1.0, 1.0]
 [0.0001, -1.0, 2.0, 0.5]
 [0.0001, -1.0, 2.0, 0.5]
 [0.0001, -1.0, 2.0, 0.5]
 [0.003, -1.0, 1.0, 1.0]
 [0.003, -1.0, 1.0, 1.0]
 [0.003, -1.0, 1.0, 1.0]
 [0.003, -1.0, 2.0, 0.5]
 ⋮
 [0.08, -1.0, 2.0, 0.5]
 [0.08, -1.0, 2.0, 0.5]
 [0.08, -1.0, 2.0, 0.5]
 [0.08, -1.0, 4.0, 0.125]
 [0.08, -1.0, 4.0, 0.125]
 [0.08, -1.0, 4.0, 0.125]
 [0.08, 1.0, 4.0, 0.125]
 [0.08, 1.0, 4.0, 0.125]
 [0.08, 1.0, 4.0, 0.125]

In [27]:
function adjust_symm_val!(g_value,θ_val,exp_part,tpz)
    g_value +=exp_part*θ_val*tpz
    return g_value
end
function adjust_symm_val!(g_value,exp_part,θ,λ,ζ,tpz)
    θ_val = theta_part(θ,λ,ζ)
    return adjust_symm_val!(g_value,θ_val,exp_part,tpz)
end

function calc_symmetry_value(g_vec,positions,dist2_mat,f_mat,indices,η,λ,ζ,tpz)

    θ_vector = all_angular_measure(positions[indices[1]],positions[indices[2]],positions[indices[3]],dist2_mat[indices[1],indices[2]],dist2_mat[indices[1],indices[3]],dist2_mat[indices[2],indices[3]])

    exp_val = exponential_part(η,dist2_mat[indices[1],indices[2]],dist2_mat[indices[1],indices[3]],dist2_mat[indices[2],indices[3]],f_mat[indices[1],indices[2]],f_mat[indices[1],indices[3]],f_mat[indices[2],indices[3]])

    for (θ,index) in zip(θ_vector,indices)
        g_vec[index] = adjust_symm_val!(g_vec[index],exp_val,θ,λ,ζ,tpz)
    end

    return g_vec
end
function calc_symmetry_values(positions,dist2_mat,f_mat,g_vec,symmetry_function::AngularType3)
    N = length(g_vec)  
    if symmetry_function.type_vec == [1.,1.,1.]
        η,λ,ζ,tpz = symmetry_function.eta,symmetry_function.lambda,symmetry_function.zeta,symmetry_function.tpz
        for atomindex in eachindex(g_vec)
            for index2 in (atomindex+1):N
                for index3 in (index2+1):N
                    g_vec = calc_symmetry_value(g_vec,positions,dist2_mat,f_mat,[atomindex,index2,index3],η,λ,ζ,tpz )
                end
            end
        end
    else
        g_vec = zeros(N)
    end


    return g_vec
end
function calc_symmetry_values(positions,dist2_mat,f_mat,g_vec,symmetry_function::RadialType2)
    return calc_symm_vals!(positions,dist2_mat,f_mat,g_vec,symmetry_function)
end

function total_calculation(positions,dist2_mat,f_mat,total_symm_vec)
    
    g_mat = zeros(88,55)

    for g_index in eachindex(total_symm_vec)

        g_mat[g_index,:] = calc_symmetry_values(positions,dist2_mat,f_mat,g_mat[g_index,:],total_symm_vec[g_index])

    end
    return g_mat
end

total_calculation (generic function with 1 method)

In [28]:
g_vec_test = calc_symmetry_values(positions,dis2mat,f_mat,zeros(55),angularsymmvec[1]) 

g_vec_test ≈ truevals[11,:]

true

In [29]:
@benchmark calc_symmetry_values($positions,$dis2mat,$f_mat,$zeros(55),$angularsymmvec[1])

BenchmarkTools.Trial: 3142 samples with 1 evaluation.
 Range (min … max):  1.165 ms … 7.224 ms  ┊ GC (min … max):  0.00% … 72.08%
 Time  (median):     1.278 ms             ┊ GC (median):     0.00%
 Time  (mean ± σ):   1.588 ms ± 1.092 ms  ┊ GC (mean ± σ):  13.82% ± 15.49%

  ██▄▃▂   ▂▄                                                ▁
  ███████▇██▇▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆█▇████▇ █
  1.17 ms     Histogram: log(frequency) by time     6.66 ms <

 Memory estimate: 4.00 MiB, allocs estimate: 52472.

In [30]:
total_calculation(positions,dis2mat,f_mat,total_symm_vec) ≈ truevals

true

In [31]:
@benchmark total_calculation($positions,$dis2mat,$f_mat,$total_symm_vec)

BenchmarkTools.Trial: 118 samples with 1 evaluation.
 Range (min … max):  40.144 ms … 50.542 ms  ┊ GC (min … max): 11.43% … 9.09%
 Time  (median):     41.290 ms              ┊ GC (median):    11.35%
 Time  (mean ± σ):   42.550 ms ±  2.641 ms  ┊ GC (mean ± σ):  13.42% ± 3.57%

     ▄ ▄█▂▃▁                                                   
  ▅▃▅█▆█████▃▅▅▄▃▃▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▄▁▁▃▅▅▁▃▆▃▆▃▃▃▁▁▁▁▄ ▃
  40.1 ms         Histogram: frequency by time        48.7 ms <

 Memory estimate: 104.19 MiB, allocs estimate: 1364455.

# The delta version

In [32]:
function adjust_symm_val!(g_value,θ_new,θ_old,exp_new,exp_old,tpz)
    g_value = adjust_symm_val!(g_value,θ_new,exp_new,tpz)
    g_value = adjust_symm_val!(g_value,θ_old,-exp_old,tpz)
    return g_value
end
function adjust_symm_val!(g_value,exp_old,exp_new,θ_old,θ_new,λ,ζ,tpz)
    θ_val_old,θ_val_new = theta_part(θ_old,λ,ζ),theta_part(θ_new,λ,ζ)
    return adjust_symm_val!(g_value,θ_val_new,θ_val_old,exp_new,exp_old,tpz)
end
function adjust_radial_symm_val(g_value1,g_value2,rnew_ij,r2_ij,fnew_ij,f2_ij,η)
    
    delta_g = calc_one_symm_val(rnew_ij,fnew_ij,η) - calc_one_symm_val(r2_ij,f2_ij,η)
    
    g_value1 += delta_g
    g_value2 += delta_g
    return g_value1,g_value2
end

function calc_new_symmetry_value(g_vector,indices,newpos,position1,position2,position3,rnew_ij,rnew_ik,r2_ij,r2_ik,r2_jk,fnew_ij,fnew_ik,f_ij,f_ik,f_jk,η,λ,ζ,tpz)

    θ_old_vec,θ_new_vec = all_angular_measure(newpos,position2,position3,rnew_ij,rnew_ik,r2_jk),all_angular_measure(position1,position2,position3,r2_ij,r2_ik,r2_jk)
    exp_old,exp_new = exponential_part(η,rnew_ij,rnew_ik,r2_jk,fnew_ij,fnew_ik,f_jk),exponential_part(η,r2_ij,r2_ik,r2_jk,f_ij,f_ik,f_jk)

    for (θ_old,θ_new,index) in zip(θ_old_vec,θ_new_vec,indices)
        g_vector[index] = adjust_symm_val!(g_vector[index],exp_old,exp_new,θ_old,θ_new,λ,ζ,tpz)
    end

    return g_vector
end
function calc_new_symmetry_value(g_vector,indexi,indexj,indexk,newpos,positions,dist2_mat,new_dist2_vec,f_mat,new_f_vec,η,λ,ζ,tpz )

    return calc_new_symmetry_value(g_vector,[indexi,indexj,indexk],newpos,positions[indexi],positions[indexj],positions[indexk],new_dist2_vec[indexj],new_dist2_vec[indexk], dist2_mat[indexi,indexj],dist2_mat[indexi,indexk],dist2_mat[indexj,indexk],new_f_vec[indexj],new_f_vec[indexk],f_mat[indexi,indexj],f_mat[indexi,indexk],f_mat[indexj,indexk],η,λ,ζ,tpz)
end
function calc_new_symmetry_value(g_vector,indexi,indexj,positions,newpos,dist2_mat,new_dist_vec,f_mat,new_f_vec,η)
    g_vector[indexi],g_vector[indexj] = adjust_radial_symm_val(g_vector[indexi],g_vector[indexj],new_dist_vec[indexj],dist2_mat[indexi,indexj],new_f_vec[indexj],f_mat[indexi,indexj],η)
    
    return g_vector
end

calc_new_symmetry_value (generic function with 3 methods)

In [33]:
calc_new_symmetry_value(zeros(55),3,2,1,newpos,positions,dis2mat,new_dis_vec,f_mat,new_f_vec,angularsymmvec[1].eta,angularsymmvec[1].lambda,angularsymmvec[1].zeta,angularsymmvec[1].tpz )

55-element Vector{Float64}:
 -0.06368813970058035
 -0.04737292664180842
 -0.029440145758675762
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [34]:
function symm_calculation(g_vector,atomindex,newpos,positions,dist2_mat,new_dis_vec,f_mat,new_f_vec,symmetry_function::RadialType2)

    if symmetry_function.type_vec == [1.,1.]
        η = symmetry_function.eta
        for index2 in eachindex(g_vector)
            if index2 != atomindex
                g_vector = calc_new_symmetry_value(g_vector,atomindex,index2,positions,newpos,dist2_mat,new_dis_vec,f_mat,new_f_vec,η)
            end
        end
    end
    return g_vector
end
function symm_calculation(g_vector,atomindex,newpos,positions,dist2_mat,new_dis_vec,f_mat,new_f_vec,symmetry_function::AngularType3)

    N = length(g_vector)  
    if symmetry_function.type_vec == [1.,1.,1.]
        η,λ,ζ,tpz = symmetry_function.eta,symmetry_function.lambda,symmetry_function.zeta,symmetry_function.tpz
        for j_index in 1:N 
            if j_index != atomindex
                for k_index in j_index+1:N
                    if k_index != atomindex
                        g_vector = calc_new_symmetry_value(g_vector,atomindex,j_index,k_index,newpos,positions,dist2_mat,new_dis_vec,f_mat,new_f_vec,η,λ,ζ,tpz)
                    end
                end
            end
        end
    end

    return g_vector
end

symm_calculation(zeros(55),3,newpos,positions,dis2mat,new_dis_vec,f_mat,new_f_vec,angularsymmvec[1])

@benchmark symm_calculation($zeros(55),$3,$newpos,$positions,$dis2mat,$new_dis_vec,$f_mat,$new_f_vec,$angularsymmvec[1])

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  174.910 μs …   5.361 ms  ┊ GC (min … max):  0.00% … 94.12%
 Time  (median):     191.976 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   253.710 μs ± 446.937 μs  ┊ GC (mean ± σ):  16.26% ±  8.81%

  ▅▇██▇▆▆▆▅▅▄▄▃▂▂▁▁                       ▁▂▂▃▂▃▂▂▁▁▁           ▂
  █████████████████████████▇▇▇▇█▇▆▆▆▇▇▇▆▆▇████████████▇█▆▅▇▆▅▅▄ █
  175 μs        Histogram: log(frequency) by time        393 μs <

 Memory estimate: 783.16 KiB, allocs estimate: 10020.

In [35]:
function total_symm_(g_mat,positions,new_pos,dist2_mat,new_dist_vec,f_mat,new_f_vec,atomindex,total_symm_vec)
    for g_index in eachindex(total_symm_vec)
        g_mat[g_index,:] = symm_calculation(g_mat[g_index,:],atomindex,newpos,positions,dist2_mat,new_dis_vec,f_mat,new_f_vec,total_symm_vec[g_index])
    end
    return g_mat
end

total_symm_ (generic function with 1 method)

In [36]:
truevals

88×55 transpose(::Matrix{Any}) with eltype Any:
 14.6407       11.9666       11.9666       …  7.45537      5.69954
  0.0           0.0           0.0             0.0          0.0
  7.16978       6.36224       6.36224         3.92888      2.94156
  0.0           0.0           0.0             0.0          0.0
  4.52421       4.14311       4.14311         2.54596      1.91555
  0.0           0.0           0.0          …  0.0          0.0
  0.972533      0.857168      0.857168        0.508399     0.410771
  0.0           0.0           0.0             0.0          0.0
  0.00179489    0.00117603    0.00117603      0.000528536  0.000557398
  0.0           0.0           0.0             0.0          0.0
  ⋮                                        ⋱               
  0.00742548    0.00500205    0.00500205      0.00192165   0.00165975
  0.0           0.0           0.0          …  0.0          0.0
  0.0           0.0           0.0             0.0          0.0
  0.000947854   0.000598351   0.000598351

In [40]:
xmat = total_calculation(positions,dis2mat,f_mat,total_symm_vec)

88×55 Matrix{Float64}:
 14.6407        11.9666        11.9666      …     7.45537         5.69954
  0.0            0.0            0.0               0.0             0.0
  7.16978        6.36224        6.36224           3.92888         2.94156
  0.0            0.0            0.0               0.0             0.0
  4.52421        4.14311        4.14311           2.54596         1.91555
  0.0            0.0            0.0         …     0.0             0.0
  0.972533       0.857168       0.857168          0.508399        0.410771
  0.0            0.0            0.0               0.0             0.0
  0.00179489     0.00117603     0.00117603        0.000528536     0.000557398
  0.0            0.0            0.0               0.0             0.0
  ⋮                                         ⋱                  
 26.9775      -274.288       -274.288          -719.154        -861.241
  0.0            0.0            0.0         …     0.0             0.0
  0.0            0.0            0.0           

In [22]:
@benchmark total_symm_($g_mat,$positions,$newpos,$dis2mat,$new_dis_vec,$f_mat,$new_f_vec,$3,$total_symm_vec)

BenchmarkTools.Trial: 724 samples with 1 evaluation.
 Range (min … max):  4.916 ms … 12.434 ms  ┊ GC (min … max):  0.00% … 41.53%
 Time  (median):     5.594 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   6.907 ms ±  2.555 ms  ┊ GC (mean ± σ):  15.94% ± 17.55%

   ▃█▄▁▁▂                                                     
  ▃██████▆▃▂▃▃▂▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▃▃▃▃▂▂▂▂▂▃▃▃▄▄▅▃▄ ▃
  4.92 ms        Histogram: frequency by time        12.2 ms <

 Memory estimate: 19.92 MiB, allocs estimate: 260618.